In [1]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np

In [2]:
fp_df = pd.read_csv('../iss-data/csv/iss_flight_plan_20220101-20251231.csv')

In [3]:
launch_dates_df = fp_df.loc[fp_df['event'] == 'Launch']

In [4]:
launch_dates_df = launch_dates_df.drop(['port_name', 'vehicle_type', 'eva_name', 'eva_type', 'eva_accuracy', 'event'], axis=1)

In [5]:
launch_dates = launch_dates_df['datedim'].to_numpy()

In [6]:
vehicle_names = launch_dates_df['vehicle_name'].to_numpy()

We need the two above variables. But we especially need the `launch_dates` array for the optimization function.

In [7]:
thresholds_df = pd.read_csv('../iss-data/csv/thresholds_limits_definition.csv')

In [8]:
thresholds_df

,threshold_category,threshold_value,threshold_owner,units
0,ACY Insert,800.0,USOS,ACY Inserts
1,KTO,22.0,USOS,KTO
2,Pretreat Tanks,4.0,USOS,Pretreat Tanks
3,Filter Inserts,4.0,USOS,Filter Inserts
4,Urine Receptacle,4.0,USOS,Urine Receptacles
5,EDVs,21.0,USOS,EDVs
6,ACY Insert,239.0,RSOS,ACY Inserts
7,KTO,6.0,RSOS,KTO
8,Pretreat Tanks,1.0,RSOS,Pretreat Tanks
9,Filter Inserts,4.0,RSOS,Filter Inserts


In [9]:
thresholds_df = thresholds_df.loc[thresholds_df['threshold_owner'] != 'RSOS']

In [10]:
thresholds_df = thresholds_df.loc[thresholds_df['threshold_category'] != 'Water Critical']

In [11]:
thresholds_df

,threshold_category,threshold_value,threshold_owner,units
0,ACY Insert,800.0,USOS,ACY Inserts
1,KTO,22.0,USOS,KTO
2,Pretreat Tanks,4.0,USOS,Pretreat Tanks
3,Filter Inserts,4.0,USOS,Filter Inserts
4,Urine Receptacle,4.0,USOS,Urine Receptacles
5,EDVs,21.0,USOS,EDVs
11,O2 (Oxygen),402.0,NaN,lbs
12,N2 (Nitrogen),275.0,NaN,lbs
13,Water Alert,1088.0,USOS,Liters
15,Food,160.0,USOS,BOB


In [12]:
thresholds_df.loc[0, 'threshold_category'] = 'ACY Inserts'
thresholds_df.loc[11, 'threshold_category'] = 'Oxygen'
thresholds_df.loc[12, 'threshold_category'] = 'Nitrogen'
thresholds_df.loc[13, 'threshold_category'] = 'Water'
thresholds_df.loc[15, 'threshold_category'] = 'US Food BOBs'
thresholds_df = thresholds_df.reindex()

In [13]:
units = thresholds_df['units'].tolist()

In [14]:
thresholds = thresholds_df['threshold_value'].to_numpy()

In [15]:
consumables_str = thresholds_df['threshold_category'].to_numpy()
consumable_id_mapping = {consumable: i for i, consumable in enumerate(consumables_str)}
consumables_required = np.array([list(consumable_id_mapping.values())])
usage_rates = np.array([3.9, 0.107142738, 0.016666668, 0.22727274, 0.22727274, 0.22727274, 5.49, 0.484929, 20.02, 0.081])
launch_dates = np.array(launch_dates)

In [16]:
print(f"launch_dates: {launch_dates}")
print(f"thresholds: {thresholds}")
print(f"consumables_str: {consumables_str}")
print(f"consumables_required: {consumables_required}")
print(f"id_mapping: {consumable_id_mapping}")
print(f"consumables_required_numeric: {consumables_required}")

launch_dates: ['2/15/2022' '2/19/2022' '4/8/2022' '5/19/2022' '7/15/2022' '10/5/2022'
 '10/26/2022' '11/7/2022' '11/26/2022' '2/9/2023' '2/24/2023' '3/2/2023'
 '3/15/2023' '5/21/2023' '6/5/2023' '8/2/2023' '8/23/2023' '8/26/2023'
 '11/1/2023' '1/10/2024' '1/17/2024' '1/29/2024' '2/15/2024' '2/18/2024'
 '3/4/2024' '4/11/2024' '5/1/2024' '5/10/2024' '5/31/2024' '6/30/2024'
 '8/4/2024' '8/19/2024' '8/25/2024' '9/15/2024' '11/1/2024' '12/3/2024'
 '12/7/2024' '12/30/2024' '12/31/2024' '2/1/2025' '2/6/2025' '2/24/2025'
 '5/15/2025' '5/16/2025' '6/14/2025' '8/1/2025' '8/4/2025' '9/1/2025'
 '10/1/2025' '11/1/2025' '11/26/2025' '12/20/2025']
thresholds: [ 800.   22.    4.    4.    4.   21.  402.  275. 1088.  160.]
consumables_str: ['ACY Inserts' 'KTO' 'Pretreat Tanks' 'Filter Inserts' 'Urine Receptacle'
 'EDVs' 'Oxygen' 'Nitrogen' 'Water' 'US Food BOBs']
consumables_required: [[0 1 2 3 4 5 6 7 8 9]]
id_mapping: {'ACY Inserts': 0, 'KTO': 1, 'Pretreat Tanks': 2, 'Filter Inserts': 3, 'Urine Recept

In [17]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('GLOP')
num_launches = len(launch_dates)
num_consumables = len(consumables_required[0])

In [18]:
consumables = [[solver.NumVar(0, solver.infinity(), f"consumable_{i}_{j}") for j in range(num_consumables)] for i in range(num_launches)]

In [19]:
objective = solver.Objective()
objective.SetMinimization()

In [20]:
for i in range(num_launches):
    for j in range(num_consumables):
        objective.SetCoefficient(consumables[i][j], 1)

In [21]:
for i in range(num_consumables):
    constraint = solver.Constraint(thresholds[i], solver.infinity())
    for j in range(num_launches):
        constraint.SetCoefficient(consumables[j][i], -1)

In [22]:
usage_rate_constraint = solver.Constraint(0, 0)
for i in range(num_launches):
    for j in range(num_consumables):
        usage_rate_constraint.SetCoefficient(consumables[i][j], -usage_rates[j])

In [23]:
solver.Solve()

2

In [24]:
consumables_per_launch = np.zeros((num_launches, num_consumables))

In [ ]:
for i in range(num_launches):
    for j in range(num_consumables):
        consumables_per_launch[i, j] = consumables[i][j].solution_value()

In [ ]:
print("Optimal number of consumables in each launch:")
print(consumables_per_launch)
print("Total number of launches:", int(objective.Value()))